The core idea of agents is to use a language model to choose a sequence of actions to take.
In chains, a sequence of actions is hardcoded (in code).
In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

## [Quick Start](./quick_start)

For a quick start to working with agents, please check out [this getting started guide](./quickstart). This covers basics like initializing an agent, creating tools, and adding memory.

## [Concepts](./concepts)

There are several key concepts to understand when building agents: Agents, AgentExecutor, Tools, Toolkits.
For an in depth explanation, please check out [this conceptual guide](./concepts)


## [Agent Types](./agent_types)

There are many different types of agents to use. For a overview of the different types and when to use them, please check out [this section](./agent_types).

## [How To Guides](./how_to)

Agents have a lot of related functionality! Check out comprehensive guides including:

- [Building a custom agent](./how_to/custom_agent)
- [Streaming (of both intermediate steps and tokens](./how_to/streaming)
- ... and [much much more!](./how_to)

## [Tools](./tools)

Agents are only as good as the tools they have. For a comprehensive guide on tools, please see [this section](./tools).

